#### Notes

This code takes a main folder with subsubsolders where the labels per frame are stored. The folder tree looks, for example, like:

Sweden:\
-Channel_1\
--Video_1\
---labels\
----label_1.txt\
----label_2.txt\
----...\
--Video_2\
---labels\
----label_1.txt\
...

* Some of the labels directories are empty 

In [ ]:
import os
import pandas as pd
import parquet 
from collections import defaultdict
import os.path
from fastparquet import write

In [ ]:
main_folder = 'path/to/main'

def result_processing(path_to_results:str):
    ### This will return a parque file 
    
    file_path = 'detection_results.parquet'
    
    # Channel names 
    channel_dirs = [filename for filename in os.listdir(main_folder) if not filename.startswith('.')]
    for channel_d in channel_dirs:
        sub_folder_path = os.path.join(main_folder,channel_d)
        # Access the video's frames folder 
        videos_dir = [filename for filename in os.listdir(sub_folder_path) if not filename.startswith('.')]
        for video_d in videos_dir:
            video_path = os.path.join(sub_folder_path,video_d)
            labels_dir_path =  os.path.join(video_path,'/labels/')
            
            if os.listdir():#Labels dir is not empty:
                
                # Access labels information
            
                # Save detection information in parque file 
                info = {'Channel':[channel_d],
                        'Video_id':[video_d], 
                        'class_label':,
                        'x_center':,
                        'y_center':,
                        'width':,
                        'height':,
                       } 
                # If the file does not exists, create it
                if not os.path.isfile(file_path):
                    write(file_path, pd.DataFrame(info))
                else: # Otherwise, write on it
                    write(file_path, pd.DataFrame(info), append=True)
            else: 
                pass

def transf_parque(parque_file):
    ####